### Method 1

1. upload all the files to the google colab storage session
2. using the sqldf library to run sql queries

In [24]:
!pip install sqldf

In [25]:
#import libraries
import sqldf
import pandas as pd

#use pandas to import the file
#the path is from the google colab storage session
circuits = pd.read_csv('/content/circuits.csv')
circuits.head(2)


#run the query
query = """
SELECT *
FROM circuits
LIMIT 5

"""

sqldf.run(query)

,index,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [37]:
#demonstrating joins with sqldf
#for some reason, aliases don't seem to work

constructors = pd.read_csv('/content/constructors.csv')
constructors_standings = pd.read_csv('/content/constructor_standings.csv')

query = """
SELECT constructors.name,
       SUM(constructors_standings.points) AS total_points
FROM constructors
LEFT JOIN constructors_standings   ON
  constructors.constructorId = constructors_standings.constructorId
GROUP BY constructors.name
ORDER BY total_points DESC
LIMIT 10
"""

sqldf.run(query)

,name,total_points
0,Ferrari,91389.0
1,Mercedes,73379.5
2,Red Bull,68243.5
3,McLaren,57557.5
4,Williams,32147.5
5,Renault,17168.0
6,Force India,10347.0
7,Team Lotus,7742.0
8,Lotus F1,7485.0
9,Benetton,7349.5


### Method 2

Using sqlite3 + pandas interface

In [38]:
#import library
import sqlite3

In [39]:
#create connection
conn = sqlite3.connect('/content/database.db')

In [41]:
query = """
SELECT *
FROM circuits
LIMIT 5

"""
pd.read_sql(query, conn)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [43]:
results = pd.read_sql(query, conn)

pandas.core.frame.DataFrame